# Fine-tuning Embeddings for RAG on Specific Data



## Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API and RAGAS Key.

### Nest Asyncio

In [5]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

In [2]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.

In [3]:
!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 15.9 MB/s eta 0:00:00


In [4]:
!pip install -qU ragas==0.2.10 rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00


In [6]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 45.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.


In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")
os.environ["RAGAS_APP_TOKEN"] = getpass.getpass("Please enter your Ragas API key!")

Enter Your OpenAI API Key: ··········
Please enter your Ragas API key!··········


## Loading Data

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are few of the cricket blogs.

- [At 36, Jadeja is as good as they get](https://divyanshpeswani.substack.com/p/at-36-jadeja-is-as-good-as-they-get)
- [The lines and lengths are trying to tell us something](https://divyanshpeswani.substack.com/p/the-lines-and-lengths-are-trying)
- [What makes a successful run chase in the IPL](https://divyanshpeswani.substack.com/p/what-makes-a-successful-run-chase)
- [Do teams even play ODIs anymore?](https://www.goodareas.co/p/do-teams-even-play-odis-anymore)
- [Bazball's mission to Moscow](https://www.goodareas.co/p/bazballs-mission-to-moscow)
- [Who is the greatest active fast bowler in Test cricket?](https://www.goodareas.co/p/who-is-the-greatest-active-fast-bowler-in-test-cricket)

Let's start by collecting our data into a useful pile!



In [8]:
!mkdir data

In [9]:
!curl https://divyanshpeswani.substack.com/p/at-36-jadeja-is-as-good-as-they-get -o data/at-36-jadeja-is-as-good-as-they-get.html
!curl https://divyanshpeswani.substack.com/p/the-lines-and-lengths-are-trying -o data/the-lines-and-lengths-are-trying.html
!curl https://divyanshpeswani.substack.com/p/what-makes-a-successful-run-chase -o data/what-makes-a-successful-run-chase.html
!curl https://www.goodareas.co/p/do-teams-even-play-odis-anymore -o data/do-teams-even-play-odis-anymore.html
!curl https://www.goodareas.co/p/bazballs-mission-to-moscow -o data/bazballs-mission-to-moscow.html
!curl https://www.goodareas.co/p/who-is-the-greatest-active-fast-bowler-in-test-cricket -o data/who-is-the-greatest-active-fast-bowler-in-test-cricket.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  179k    0  179k    0     0   236k      0 --:--:-- --:--:-- --:--:--  236k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  214k    0  214k    0     0   403k      0 --:--:-- --:--:-- --:--:--  402k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k    0  197k    0     0   349k      0 --:--:-- --:--:-- --:--:--  350k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  269k    0  269k    0     0   425k      0 --:--:-- --:--:-- --:--:--  425k
  % Total    % Received % Xferd  Average Speed   Tim

Load into langchain document object

In [10]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import BSHTMLLoader

path = "data/"
text_loader = DirectoryLoader(path, glob="*.html", loader_cls=BSHTMLLoader)

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

In [12]:
training_documents = text_splitter.split_documents(text_loader.load())

In [13]:
len(training_documents)

119

Associate unique id to each document

In [14]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [15]:
training_split_percentage = 24
val_split_percentage = 12
test_split_percentage = 64

train_split_len = int(len(training_documents) * training_split_percentage / 100)
val_split_len = int(len(training_documents) * val_split_percentage / 100)
test_split_len = len(training_documents) - train_split_len - val_split_len

In [16]:
training_split_documents = training_documents[:train_split_len]
test_split_documents = training_documents[train_split_len:train_split_len+test_split_len]
val_split_documents = training_documents[train_split_len+test_split_len:]

##Constructing a Fine-tuning Dataset

We would two approaches to construct fine tuning dataset.

1.   Naive approach where:
 * We look at a document
 * We generate questions that could be answered by that node
2.   RAGAS Knowledge graph approach

We would be utilizing OpenAI's `gpt-4o-mini`


### 1. Naive approach for constructing Fine-tuning Dataset

In [17]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [18]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [19]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [20]:
import tqdm

async def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}
  for document in tqdm.tqdm(documents):
    response = await question_generation_chain.ainvoke({
        "context":document.page_content,
        "n_questions":n_questions
      }
    )
    questions_generated = response.content.split("\n")
    for question in questions_generated:
      question_id = str(uuid.uuid4())
      questions[question_id] = "".join(question.split(".")[1:]).strip()
      relevant_docs[question_id] = [document.metadata["id"]]

  return questions, relevant_docs

In [21]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)

100%|██████████| 28/28 [00:35<00:00,  1.25s/it]


In [22]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

100%|██████████| 77/77 [01:36<00:00,  1.25s/it]


In [23]:
len(training_questions)

56

#### Reformating and Saving Datasets


In [24]:
import json

def saveDataSet(name,docs,questions,revelant_docs):
  corpus = {doc.metadata["id"]:doc.page_content for doc in docs}

  dataset = {
      "questions" : questions,
      "relevant_contexts" : revelant_docs,
      "corpus" : corpus
  }

  with open(name, "w") as f:
    json.dump(dataset, f)

  return dataset

In [25]:
training_dataset = saveDataSet("data/training_dataset.jsonl",training_split_documents,training_questions,training_relevant_contexts)
val_dataset = saveDataSet("data/val_dataset.jsonl",val_split_documents,val_questions,val_relevant_contexts)
test_dataset = saveDataSet("data/test_dataset.jsonl",test_split_documents,test_questions,test_relevant_contexts)

### 2. RAGAS Knowledge Graph Approach

Define persona's for the data (cricket)

In [26]:
from ragas.testset.persona import Persona

persona_analyst = Persona(
    name="Analyst",
    role_description="You are an expert cricket analyst",
)
persona_player = Persona(
    name="Player",
    role_description="You are an experienced cricket player",
)
persona_viewer = Persona(
    name="Viewer",
    role_description="You are an ardent cricket fan, who keeps up with latest news",
)

personas = [persona_analyst, persona_player, persona_viewer]
personas

[Persona(name='Analyst', role_description='You are an expert cricket analyst'),
 Persona(name='Player', role_description='You are an experienced cricket player'),
 Persona(name='Viewer', role_description='You are an ardent cricket fan, who keeps up with latest news')]

In [27]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

from ragas.testset import TestsetGenerator

kg_generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings,persona_list=personas)



Reformat the training_document to include id in the content. <br/>
RAGAS Knowledge Graph testset generator doesn't return the document metadata information, hence I am prepending the id to the document content.

In [28]:
import copy
kg_training_split_documents = copy.deepcopy(training_split_documents)

for document in kg_training_split_documents:
  id = document.metadata["id"]
  document.page_content = id+"####"+document.page_content

kg_val_split_documents = copy.deepcopy(val_split_documents)

for document in kg_val_split_documents:
  id = document.metadata["id"]
  document.page_content = id+"####"+document.page_content

In [29]:
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1)
]

In [30]:
def create_kg_questions(doc,size):
  kg_dataset = kg_generator.generate_with_langchain_docs(doc, testset_size=size,query_distribution=query_distribution)
  df = kg_dataset.to_pandas()
  questions_generated = df["user_input"].to_list()
  questions = {}
  relevant_docs = {}
  for id, question in enumerate(questions_generated):
    question_id = str(uuid.uuid4())
    questions[question_id] = question
    relevant_docs[question_id] = [df["reference_contexts"][id][0].split("####")[0]]

  return questions, relevant_docs

In [31]:
training_kg_questions, training_kg_relevant_contexts = create_kg_questions(kg_training_split_documents, 50)



Applying SummaryExtractor:   0%|          | 0/25 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/28 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/81 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/34 [00:00<?, ?it/s]

In [32]:
val_kg_questions, val_kg_relevant_contexts = create_kg_questions(kg_val_split_documents, 50)

Applying SummaryExtractor:   0%|          | 0/13 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/14 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/41 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/52 [00:00<?, ?it/s]

In [33]:
def saveKGDataSet(name,docs,questions,revelant_docs):
  corpus = {doc.metadata["id"]:doc.page_content.split("####")[1] for doc in docs}

  dataset = {
      "questions" : questions,
      "corpus": corpus,
      "relevant_contexts" : revelant_docs,
  }

  with open(name, "w") as f:
    json.dump(dataset, f)

  return dataset

In [34]:
training_kg_dataset = saveKGDataSet("data/training_kg_dataset.jsonl",kg_training_split_documents,training_kg_questions,training_kg_relevant_contexts)
val_kg_dataset = saveKGDataSet("data/val_kg_dataset.jsonl",kg_val_split_documents,val_kg_questions,val_kg_relevant_contexts)

## Fine-tuning `snowflake-arctic-embed-l`

Now that we have dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-l`](https://huggingface.co/Snowflake/snowflake-arctic-embed-l) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!


In [35]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [36]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [37]:
BATCH_SIZE = 10

In [38]:
EPOCHS = 10

Loss function

In [39]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

Loader and Evaluator

In [40]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

def getLoaderEvaluator(train_dataset,val_dataset):
  corpus = train_dataset['corpus']
  queries = train_dataset['questions']
  relevant_docs = train_dataset['relevant_contexts']

  examples = []
  for query_id, query in queries.items():
      doc_id = relevant_docs[query_id][0]
      text = corpus[doc_id]
      example = InputExample(texts=[query, text])
      examples.append(example)

  loader = DataLoader(
      examples, batch_size=BATCH_SIZE
  )

  val_corpus = val_dataset['corpus']
  val_queries = val_dataset['questions']
  val_relevant_docs = val_dataset['relevant_contexts']

  evaluator = InformationRetrievalEvaluator(val_queries, val_corpus, val_relevant_docs)

  return loader, evaluator


In [41]:
naive_loader, naive_evaluator = getLoaderEvaluator(training_dataset,val_dataset)
kg_loader, kg_evaluator = getLoaderEvaluator(training_kg_dataset,val_kg_dataset)

In [42]:
import wandb
wandb.init(mode="disabled")

In [46]:
from huggingface_hub import notebook_login

notebook_login()

hf_username = "ashwinpatti"

In [44]:
def fineTune(name,loader,evaluator):
  warmup_steps = int(len(loader) * EPOCHS * 0.1)

  model.fit(
      train_objectives=[(loader, train_loss)],
      epochs=EPOCHS,
      warmup_steps=warmup_steps,
      output_path=name,
      show_progress_bar=True,
      evaluator=evaluator,
      evaluation_steps=50
  )
  model.push_to_hub(f"{hf_username}/{name}-legal-ft-v0")

In [47]:
fineTune("finetuned_arctic_naive_ft",naive_loader,naive_evaluator)
fineTune("finetuned_arctic_kg_ft",kg_loader,kg_evaluator)

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
6,No log,No log,0.750000,0.892857,1.000000,1.000000,0.750000,0.297619,0.200000,0.100000,0.750000,0.892857,1.000000,1.000000,0.873794,0.832738,0.832738
12,No log,No log,0.714286,0.857143,1.000000,1.000000,0.714286,0.285714,0.200000,0.100000,0.714286,0.857143,1.000000,1.000000,0.855026,0.807738,0.807738
18,No log,No log,0.714286,0.857143,1.000000,1.000000,0.714286,0.285714,0.200000,0.100000,0.714286,0.857143,1.000000,1.000000,0.855026,0.807738,0.807738
24,No log,No log,0.678571,0.821429,1.000000,1.000000,0.678571,0.273810,0.200000,0.100000,0.678571,0.821429,1.000000,1.000000,0.844046,0.792857,0.792857
30,No log,No log,0.678571,0.857143,1.000000,1.000000,0.678571,0.285714,0.200000,0.100000,0.678571,0.857143,1.000000,1.000000,0.846521,0.795833,0.795833
36,No log,No log,0.678571,0.857143,1.000000,1.000000,0.678571,0.285714,0.200000,0.100000,0.678571,0.857143,1.000000,1.000000,0.846521,0.795833,0.795833
42,No log,No log,0.678571,0.857143,1.000000,1.000000,0.678571,0.285714,0.200000,0.100000,0.678571,0.857143,1.000000,1.000000,0.846521,0.795833,0.795833
48,No log,No log,0.678571,0.857143,1.000000,1.000000,0.678571,0.285714,0.200000,0.100000,0.678571,0.857143,1.000000,1.000000,0.846521,0.795833,0.795833
50,No log,No log,0.678571,0.857143,1.000000,1.000000,0.678571,0.285714,0.200000,0.100000,0.678571,0.857143,1.000000,1.000000,0.846521,0.795833,0.795833
54,No log,No log,0.678571,0.857143,1.000000,1.000000,0.678571,0.285714,0.200000,0.100000,0.678571,0.857143,1.000000,1.000000,0.846521,0.795833,0.795833


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
4,No log,No log,0.442308,0.711538,0.826923,1.000000,0.442308,0.237179,0.165385,0.100000,0.442308,0.711538,0.826923,1.000000,0.703124,0.610005,0.610005
8,No log,No log,0.480769,0.730769,0.826923,1.000000,0.480769,0.243590,0.165385,0.100000,0.480769,0.730769,0.826923,1.000000,0.712322,0.622901,0.622901
12,No log,No log,0.480769,0.730769,0.807692,1.000000,0.480769,0.243590,0.161538,0.100000,0.480769,0.730769,0.807692,1.000000,0.715982,0.627205,0.627205
16,No log,No log,0.480769,0.730769,0.826923,1.000000,0.480769,0.243590,0.165385,0.100000,0.480769,0.730769,0.826923,1.000000,0.713270,0.623840,0.623840
20,No log,No log,0.480769,0.730769,0.826923,1.000000,0.480769,0.243590,0.165385,0.100000,0.480769,0.730769,0.826923,1.000000,0.715692,0.626931,0.626931
24,No log,No log,0.480769,0.750000,0.846154,1.000000,0.480769,0.250000,0.169231,0.100000,0.480769,0.750000,0.846154,1.000000,0.718897,0.630594,0.630594
28,No log,No log,0.480769,0.750000,0.846154,1.000000,0.480769,0.250000,0.169231,0.100000,0.480769,0.750000,0.846154,1.000000,0.719337,0.631052,0.631052
32,No log,No log,0.480769,0.750000,0.846154,1.000000,0.480769,0.250000,0.169231,0.100000,0.480769,0.750000,0.846154,1.000000,0.719337,0.631052,0.631052
36,No log,No log,0.480769,0.750000,0.846154,1.000000,0.480769,0.250000,0.169231,0.100000,0.480769,0.750000,0.846154,1.000000,0.719337,0.631052,0.631052
40,No log,No log,0.480769,0.750000,0.846154,1.000000,0.480769,0.250000,0.169231,0.100000,0.480769,0.750000,0.846154,1.000000,0.719337,0.631052,0.631052


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

## Evaluating our Retriever

Now that we have fine-tuned our retrievers - let's see if it's worthwhile!

We'll start with some basic imports.

In [48]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

In [49]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

We'll evaluate following models:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-l`.
3. Fine-tune with naive data generation `finetuned_arctic_naive_ft`
4. Fine-tune with Ragas KG data generation `finetuned_arctic_kg_ft`



### `text-embedding-3-small`

In [50]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 154/154 [01:01<00:00,  2.52it/s]


In [51]:
te3_results_df = pd.DataFrame(te3_results)

In [52]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

0.9415584415584416

### `Snowflake/snowflake-arctic-embed-l` (base)

In [53]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 154/154 [00:02<00:00, 53.37it/s]


In [54]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [55]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.7402597402597403

### `Snowflake/snowflake-arctic-embed-l` (fine-tuned with naive data)


In [56]:
finetune_naive_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_naive_ft")
finetune_naive_results = evaluate_openai(test_dataset, finetune_naive_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_naive_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 154/154 [00:02<00:00, 51.68it/s]


In [57]:
finetune_naive_results_df = pd.DataFrame(finetune_naive_results)

In [58]:
finetune_hit_naive_rate = finetune_naive_results_df["is_hit"].mean()
finetune_hit_naive_rate

0.961038961038961

### `Snowflake/snowflake-arctic-embed-l` (fine-tuned with RAGAS KG data)


In [59]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_kg_ft")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_kg_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 154/154 [00:03<00:00, 48.86it/s]


In [60]:
finetune_results_df = pd.DataFrame(finetune_results)

In [61]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

0.9415584415584416

### 1. Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

In [62]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
finetune_naive_embeddings = HuggingFaceEmbeddings(model_name="ashwinpatti/finetuned_arctic_naive_ft-legal-ft-v0")
finetune_ragas_kg__embeddings = HuggingFaceEmbeddings(model_name="ashwinpatti/finetuned_arctic_kg_ft-legal-ft-v0")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/32.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

#### New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [63]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(text_loader.load())

In [91]:
vibe_questions = ["What are Jarrod Kimber's view on ODI?",
                  'How has the approach to run chases in the IPL changed from 2019 to 2024?',
                  'What is bazball?',
                  'How many runs does Andre Russell typically score before losing a wicket based on his performance statistics?']

In [152]:
vibe_answers = {"snowflake-arctic-embed-l":[],
                "finetuned_arctic_naive_ft":[],
                "finetuned_arctic_kg_ft":[]}

#### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [93]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [94]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [95]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [96]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [153]:
for question in vibe_questions:
  vibe_answers["snowflake-arctic-embed-l"].append(base_rag_chain.invoke({"question" : question})["response"])


#### Fine-tuned with naive data Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [98]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_naive_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [150]:
finetune_naive_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [154]:
for question in vibe_questions:
  vibe_answers["finetuned_arctic_naive_ft"].append(finetune_naive_rag_chain.invoke({"question" : question})["response"])


#### Fine-tuned with RAGAS KG Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [101]:
finetune_kg_vectorstore = FAISS.from_documents(training_documents, finetune_ragas_kg__embeddings)
finetune_kg_retriever = finetune_kg_vectorstore.as_retriever(search_kwargs={"k": 6})

In [151]:
finetune_kg_rag_chain = (
    {"context": itemgetter("question") | finetune_kg_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [155]:
for question in vibe_questions:
  vibe_answers["finetuned_arctic_kg_ft"].append(finetune_kg_rag_chain.invoke({"question" : question})["response"])


In [156]:
data = {
    'Question':vibe_questions,
    'snowflake-arctic-embed-l':vibe_answers['snowflake-arctic-embed-l'],
    'finetuned_arctic_naive_ft':vibe_answers['finetuned_arctic_naive_ft'],
    'finetuned_arctic_kg_ft':vibe_answers['finetuned_arctic_kg_ft']
}
df = pd.DataFrame(data)

df

,Question,snowflake-arctic-embed-l,finetuned_arctic_naive_ft,finetuned_arctic_kg_ft
0,What are Jarrod Kimber's view on ODI?,Jarrod Kimber suggests that ODIs have diminish...,Jarrod Kimber expresses concern about the decl...,Jarrod Kimber expresses a view that One Day In...
1,How has the approach to run chases in the IPL ...,The approach to run chases in the IPL has chan...,The approach to run chases in the IPL has chan...,The approach to run chases in the IPL has chan...
2,What is bazball?,"Bazball refers to a style of play in cricket, ...",Bazball is a style of cricket associated with ...,Bazball is a style of cricket associated with ...
3,How many runs does Andre Russell typically sco...,Andre Russell typically scores 27 runs off eve...,Andre Russell typically scores 27 runs off eve...,Andre Russell typically scores 27 runs before ...


In [157]:
hit_rate_data = {
    'Model':["snowflake-arctic-embed-l", "finetuned_arctic_naive_ft", "finetuned_arctic_kg_ft"],
    'Hit Rate': [arctic_embed_m_hit_rate, finetune_hit_naive_rate,finetune_hit_rate]
}
hit_rate_df = pd.DataFrame(hit_rate_data)
hit_rate_df
#

,Model,Hit Rate
0,snowflake-arctic-embed-l,0.740260
1,finetuned_arctic_naive_ft,0.961039
2,finetuned_arctic_kg_ft,0.941558


### Compare and Contast between embedding models based on vibe check:

**Comparison**:

* Both fine-tuned models outperform the base snowflake-arctic-embed-l model in terms of answer relevance and accuracy.
* Both exhibit a better hit rate during the quantitative evaluation
* The `finetuned_arctic_naive_ft` model often provides more concise answers, while `finetuned_arctic_kg_ft` model offers more detailed explanations in certain cases.
* Quantitatively, `finetuned_arctic_naive_ft` model shows a very slightly better hit rate.
* `finetuned_arctic_naive_ft` appears to offer a favorable balance of accuracy and conciseness.
* The `finetuned_arctic_kg_ft` model demonstrates a greater potential for contextual understanding.

**Contrast**:
* The naive approach is simpler and computationally less expensive, while RAGAS knowledge graph approach potentially leads to a deeper understanding of domain and potentially more accurate answers.
* RAGAS one involes three different personas related to the domain with varrying expertise




### 2. RAGAS Evaluation

let's use RAGAS to provide more insight info. on how things are improving!

Generate data

In [124]:
#updating the generator_llm to gpt-4o
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
dataset_generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings,persona_list=personas)

Base model

In [127]:
base_dataset = dataset_generator.generate_with_langchain_docs(text_loader.load(), testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/6 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/6 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/21 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/44 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [128]:
base_dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/f975c5d1-b921-4bfb-8b86-e59c426151af


'https://app.ragas.io/dashboard/alignment/testset/f975c5d1-b921-4bfb-8b86-e59c426151af'

In [115]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")
os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

LangChain API Key:··········


In [145]:
for test_row in base_dataset:
  response = base_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [146]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(base_dataset.to_pandas())

In [148]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [149]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[8]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[14]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[32]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')


{'context_recall': 0.3175, 'faithfulness': 0.7282, 'factual_correctness': 0.3567, 'answer_relevancy': 0.5695, 'context_entity_recall': 0.3437, 'noise_sensitivity_relevant': 0.3130}

Fine-tuned model with naive approach

In [158]:
for test_row in base_dataset:
  response = finetune_naive_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [159]:
evaluation_dataset = EvaluationDataset.from_pandas(base_dataset.to_pandas())
custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.6766, 'faithfulness': 0.8834, 'factual_correctness': 0.4517, 'answer_relevancy': 0.8908, 'context_entity_recall': 0.5930, 'noise_sensitivity_relevant': 0.2796}

Fine-tune model with RAGAS KG approach

In [160]:
for test_row in base_dataset:
  response = finetune_kg_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [161]:
evaluation_dataset = EvaluationDataset.from_pandas(base_dataset.to_pandas())
custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.6740, 'faithfulness': 0.7609, 'factual_correctness': 0.5317, 'answer_relevancy': 0.8890, 'context_entity_recall': 0.5806, 'noise_sensitivity_relevant': 0.2414}

## Compare and contrast between three models using RAGAS evaluation

In [163]:
# prompt: Compare and contrast finetuned_arctic_naive_ft and finetuned_arctic_kg_ft

import pandas as pd

# Sample data (replace with your actual results)
data = {
    'Model': ["snowflake-arctic-embed-l", "finetuned_arctic_naive_ft", "finetuned_arctic_kg_ft"],
    'Context Recall': [0.32, 0.68, 0.67],
    'Faithfulness': [0.73, 0.88, 0.76],
    'Factual Correctness': [0.36, 0.45, 0.53],
    'Answer Relevancy': [0.57, 0.89, 0.89],
    'Context Entity Recall': [0.34, 0.59, 0.58],
    'Noise Sensitivity': [0.31, 0.28, 0.24]
}

df = pd.DataFrame(data)
df


,Model,Context Recall,Faithfulness,Factual Correctness,Answer Relevancy,Context Entity Recall,Noise Sensitivity
0,snowflake-arctic-embed-l,0.32,0.73,0.36,0.57,0.34,0.31
1,finetuned_arctic_naive_ft,0.68,0.88,0.45,0.89,0.59,0.28
2,finetuned_arctic_kg_ft,0.67,0.76,0.53,0.89,0.58,0.24


**Overall Comparison**:

* **Fine-tuned Models**: Both fine-tuned models (`finetuned_arctic_naive_ft` and `finetuned_arctic_kg_ft`) demonstrate much better performance compared to the base model ((`snowflake-arctic-embed-l`)) across all metrics. Fine-tuning clearly provides substantial improvements.
* **Naive (`finetuned_artic_naive_ft`)**: Excels in faithfulness, indicating a strong ability to generate answers grounded in the provided context.
* **KG (`finetuned_arctic_kg_ft`)**: Shows higher factual accuracy and noise robustness, suggesting a better understanding of the domain and ability to handle variations in input.

